# A Guided Tour of Ray Core: Remote Classes

[*Remote Classes*](https://docs.ray.io/en/latest/walkthrough.html#remote-classes-actors)
involve using a `@ray.remote` decorator on a class. 

This implements an [*actor*](https://patterns.eecs.berkeley.edu/?page_id=258) pattern, with properties: *stateful*, *message-passing semantics*

Actors are extremely powerful. They allow you to take a Python class and instantiate it as a microservice that can be queried from other actors and tasks and even other applications.

When you instantiate a remote Actor, a separate worker process is created on the workder node. Other Ray tasks and actors can invoke its methods on that process, mutating its internal state. When the driver exits or when an actor handle goes out of scope, the Python process terminates. Actors can also be terminated manually if needed. The examples code below show all these cases.

<img src="images/actor_and_workders.png" height="30%" width="60%">

---

First, let's start Ray…

In [2]:
import logging
import time
import ray
import random
from random import randint
import numpy as np

In [3]:
ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2022-02-02_17-56-45_397837_17436/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-02-02_17-56-45_397837_17436/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-02-02_17-56-45_397837_17436',
 'metrics_export_port': 62852,
 'node_id': 'ce20e2f46ff85027aa8cd6b4476a796198c8c68f387fdf482e5b26c3'}

## 3. Remote Class as a Stateful Actor Pattern

To start, we'll define a class and use the decorator: `@ray.remote`

Let's use Python class and convert that to a remote Actor class actor service as a Parameter Server.
This is a common example in machine learning where you may have a central Parameter server updating gradients
from other worker processes computing individual gradients. 

<img src="https://terrytangyuan.github.io/img/inblog/mpi-operator-1.png" width="40%" height="20%">

In [4]:
@ray.remote
class ParameterSever:
    def __init__(self):
        # Initialized our gradients to zero
        self.params = np.zeros(10)

    def get_params(self):
        # Return current gradients
        return self.params

    def update_params(self, grad):
        # Update the gradients 
        self.params -= grad

Define work or task as a function for a remote Worker process. This could be a machine learning task that
computes gradients and sends them to the parameter server

In [8]:
@ray.remote
def worker(ps):
    # Iterate over some epoch
    for i in range(100):
        time.sleep(1.5)  # this could be your task computing gradients
        grad = np.ones(10)
        # update the gradients in the parameter server
        ps.update_params.remote(grad)

Start our Parameter Server actor. This will be scheduled as a process on a remote Ray Worker

In [9]:
param_server = ParameterSever.remote()
param_server

Actor(ParameterSever, bc31558ed44417df6d9530d201000000)

Let's get the initial values of the parameter server

In [10]:
print(f"Initial params: {ray.get(param_server.get_params.remote())}")

Initial params: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Create Workers Nodes Computing Gradients
Let's create three separate workers as our machine learning tasks that compute gradients.
These will be scheduled as tasks on a Ray cluster.

You can use list comprehension. Quite Pythonic!

If we need more workers to scale, we can always bump them up.

In [11]:
[worker.remote(param_server) for _ in range(3)]

[ObjectRef(4ee449587774c1f0ffffffffffffffffffffffff0100000001000000),
 ObjectRef(32cccd03c567a254ffffffffffffffffffffffff0100000001000000),
 ObjectRef(480a853c2c4c6f27ffffffffffffffffffffffff0100000001000000)]

Now, let's iterate over a loop and query the Parameter Server 
as the workers are running independently and updating the gradients

In [12]:
for _i in range(20):
    print(f"Updated params: {ray.get(param_server.get_params.remote())}")
    time.sleep(1)

Updated params: [-15. -15. -15. -15. -15. -15. -15. -15. -15. -15.]
Updated params: [-18. -18. -18. -18. -18. -18. -18. -18. -18. -18.]
Updated params: [-21. -21. -21. -21. -21. -21. -21. -21. -21. -21.]
Updated params: [-21. -21. -21. -21. -21. -21. -21. -21. -21. -21.]
Updated params: [-24. -24. -24. -24. -24. -24. -24. -24. -24. -24.]
Updated params: [-27. -27. -27. -27. -27. -27. -27. -27. -27. -27.]
Updated params: [-27. -27. -27. -27. -27. -27. -27. -27. -27. -27.]
Updated params: [-30. -30. -30. -30. -30. -30. -30. -30. -30. -30.]
Updated params: [-33. -33. -33. -33. -33. -33. -33. -33. -33. -33.]
Updated params: [-33. -33. -33. -33. -33. -33. -33. -33. -33. -33.]
Updated params: [-36. -36. -36. -36. -36. -36. -36. -36. -36. -36.]
Updated params: [-39. -39. -39. -39. -39. -39. -39. -39. -39. -39.]
Updated params: [-39. -39. -39. -39. -39. -39. -39. -39. -39. -39.]
Updated params: [-42. -42. -42. -42. -42. -42. -42. -42. -42. -42.]
Updated params: [-45. -45. -45. -45. -45. -45. -

# Tree of Actors Pattern

A common pattern used in Ray libraries [Ray Tune](https://docs.ray.io/en/latest/tune/index.html), [Ray Train](https://docs.ray.io/en/latest/train/train.html), and [RLlib](https://docs.ray.io/en/latest/rllib/index.html) to train models in a parallel or conduct distributed HPO.

In this common pattern, tree of actors, a collection of workers as actors, are managed by a supervisor. For example, you want to train multiple models at the same time, while being able to checkpoint/inspect its state.

<img src="https://docs.ray.io/en/latest/_images/tree-of-actors.svg" width="40%" height="20%">

Let's implement a simple example to illustrate this pattern.

In [13]:
STATES = ["RUNNING", "DONE"]

class Model:

    def __init__(self, m:str):
        self._model = m

    def train(self):
        # do some training work here
        time.sleep(1)

# Factory function to return an instance of a model type
def model_factory(m: str):
    return Model(m)

### Create a Worker Actor

In [14]:
@ray.remote
class Worker(object):
    def __init__(self, m:str):
        # type of a model: lr, cl, or nn
        self._model = m                  
        
    def state(self) -> str:
        return random.choice(STATES)
    # Do the work for this model
    def work(self) -> None:
        model_factory(self._model).train()

### Create Supervisor Actor 

In [15]:
@ray.remote
class Supervisor:
    def __init__(self):
        # Create three Actor Workers, each by its unique model type
        self.workers = [Worker.remote(name) for name in ["lr", "cl", "nn"]]
                        
    def work(self):
        # do the work 
        [w.work.remote() for w in self.workers]
        
    def terminate(self):
        [ray.kill(w) for w in self.workers]
        
    def state(self):
        return ray.get([w.state.remote() for w in self.workers])

Create a Actor instance for supervisor and launch its workers

In [16]:
sup = Supervisor.remote()

# Launch remote actors as workers
sup.work.remote()

ObjectRef(38db0ab51c6b6cfb883b3211c3c042dc78dbdab10100000001000000)

### Look at the Ray Dashboard
http://127.0.0.1:8265

In [17]:
# check their status
while True:
    # Fetch the states of all its workers
    states = ray.get(sup.state.remote())
    print(states)
    # check if all are DONE
    result = all('DONE' == e for e in states)
    if result:
        # Note: Actor processes will be terminated automatically when the initial actor handle goes out of scope in Python. 
        # If we create an actor with actor_handle = ActorClass.remote(), then when actor_handle goes out of scope and is destructed, 
        # the actor process will be terminated. Note that this only applies to the original actor handle created for the actor 
        # and not to subsequent actor handles created by passing the actor handle to other tasks.
        
        # kill supervisors all worker manually, only for illustrtation and demo
        sup.terminate.remote()

        # kill the supervisor manually, only for illustration and demo
        ray.kill(sup)
        break

['DONE', 'DONE', 'RUNNING']
['RUNNING', 'DONE', 'DONE']
['RUNNING', 'DONE', 'RUNNING']
['RUNNING', 'RUNNING', 'RUNNING']
['DONE', 'DONE', 'RUNNING']
['DONE', 'RUNNING', 'RUNNING']
['DONE', 'DONE', 'DONE']


### Passing Actor handles to Ray Tasks

You can pass actor handle instances to remote Ray tasks, which can change its 
state. The `MessageActor` keeps or clears messages, depending on the its method
invoked.

In [18]:
@ray.remote
class MessageActor(object):
    def __init__(self):
        # Keep the state of the messages
        self.messages = []
    
    def add_message(self, message):
        self.messages.append(message)
    
    # reset and clear all messages
    def get_and_clear_messages(self):
        messages = self.messages
        self.messages = []
        return messages

Define a remote function which loops around and pushes messages to the actor, having access to a handle instance as an argument. That is, we are sending it a `MessageActor` instance handle ref as an argument to it.

In [19]:
@ray.remote
def worker(message_actor, j):
    for i in range(10):
        time.sleep(1)
        message_actor.add_message.remote(
            f"Message {i} from worker {j}.")


Create a message actor.

In [20]:
message_actor = MessageActor.remote()

Start 3 tasks that push messages to the actor.

In [21]:
[worker.remote(message_actor, j) for j in range(3)]

[ObjectRef(5497aa04f981e4a1ffffffffffffffffffffffff0100000001000000),
 ObjectRef(909a212b104ea2f1ffffffffffffffffffffffff0100000001000000),
 ObjectRef(6f4f08f301901921ffffffffffffffffffffffff0100000001000000)]

Periodically get the messages and print them.

In [22]:
for _ in range(10):
    new_messages = ray.get(message_actor.get_and_clear_messages.remote())
    print("New messages\n:", new_messages)
    time.sleep(1)

New messages
: ['Message 0 from worker 1.', 'Message 0 from worker 0.', 'Message 0 from worker 2.', 'Message 1 from worker 1.', 'Message 1 from worker 0.', 'Message 1 from worker 2.', 'Message 2 from worker 1.', 'Message 2 from worker 0.', 'Message 2 from worker 2.']
New messages
: ['Message 3 from worker 1.', 'Message 3 from worker 0.', 'Message 3 from worker 2.']
New messages
: ['Message 4 from worker 1.', 'Message 4 from worker 0.', 'Message 4 from worker 2.']
New messages
: ['Message 5 from worker 1.', 'Message 5 from worker 0.', 'Message 5 from worker 2.']
New messages
: ['Message 6 from worker 1.', 'Message 6 from worker 0.', 'Message 6 from worker 2.']
New messages
: ['Message 7 from worker 1.', 'Message 7 from worker 0.', 'Message 7 from worker 2.']
New messages
: ['Message 8 from worker 0.', 'Message 8 from worker 1.', 'Message 8 from worker 2.']
New messages
: ['Message 9 from worker 1.', 'Message 9 from worker 0.', 'Message 9 from worker 2.']
New messages
: []
New messages
:

Finally, shutdown Ray

In [23]:
ray.shutdown()

---
## References

 * [Writing your First Distributed Python Application with Ray](https://www.anyscale.com/blog/writing-your-first-distributed-python-application-with-ray)
 * [Using and Programming with Actors](https://docs.ray.io/en/latest/actors.html)
 * [Advanced Patterns and Anti-Patterns in Ray](https://docs.ray.io/en/latest/ray-design-patterns/index.htmlhttps://docs.ray.io/en/latest/ray-design-patterns/index.html)